### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [255]:
# Dependencies and Setup
import pandas as pd
import numpy as np

#Number of students with passing math score
swpms = 0

#Number of students with passing reading score
swprs = 0

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39170 entries, 0 to 39169
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student ID     39170 non-null  int64 
 1   student_name   39170 non-null  object
 2   gender         39170 non-null  object
 3   grade          39170 non-null  object
 4   school_name    39170 non-null  object
 5   reading_score  39170 non-null  int64 
 6   math_score     39170 non-null  int64 
 7   School ID      39170 non-null  int64 
 8   type           39170 non-null  object
 9   size           39170 non-null  int64 
 10  budget         39170 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 3.6+ MB


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [256]:
#Create empty dataframe to store results

part1_df = pd.DataFrame(columns=["Total Schools", "Total Students", "Total Budget", "Average Math Score", 
                                 "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"])

# Calculate the total number of schools and shortening the name so it's easier to work with
sdc = school_data_complete
#print(sdc.info())
total_num_sch = len(sdc["school_name"].unique())
print(f"Total number of schools is: {total_num_sch}")

#Calculate the number of students
total_num_st = len(sdc["student_name"].unique())
#print(f"Total number of students is: {total_num_st}")

#Calculate the total budget
total_budget = '${:,.2f}'.format(sdc["budget"].sum())
#print(f"Total budget is: ${'{:,.2f}'.format(total_budget)}")

#Calculate Avg Math Score
math_score_avg = round(sdc["math_score"].mean(), 2)
#print(f"Math score average is: {math_score_avg}")

#Calculate Avg Reading Score
read_score_avg = round(sdc["reading_score"].mean(), 2)
#print(f"Math score average is: {read_score_avg}")

#Calculate the overall passing rate
opr = round((math_score_avg+read_score_avg)/2, 2)
#print(f"Overall Passing Rate is: {opr}")

#Calculate the percentage of students with a passing math score
for row in sdc["math_score"]:
    if row > 70:
        swpms += 1
per_math_pass = round(swpms/len(sdc["math_score"])*100, 2) #'{:.2%}'.format(swpms/len(sdc["math_score"]))  
swpms=0

#Calculate the percentage of students with a passing math score
for row in sdc["reading_score"]:
    if row > 70:
        swprs += 1
per_reading_pass = round(swprs/len(sdc["reading_score"]*100))  #'{:.2%}'.format(swprs/len(sdc["reading_score"]))
swprs=0

part1_df["Total Schools"] = [total_num_sch]
part1_df["Total Students"] = total_num_st
part1_df["Total Budget"] = total_budget
part1_df["Average Math Score"] = math_score_avg
part1_df["Average Reading Score"] = read_score_avg
part1_df["% Passing Math"] = per_math_pass
part1_df["% Passing Reading"] = per_reading_pass
part1_df["% Overall Passing Rate"] = opr
part1_df

Total number of schools is: 15


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,32715,"$82,932,329,558.00",78.99,81.88,72.39,1,80.44


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [278]:
#Get the count of students grouped by school
num_students = school_data_complete["school_name"].value_counts()
#print(num_students)

#get the count of passing math scores grouped by school
pass_math_s = school_data_complete[school_data_complete["math_score"] >= 70]
pms_s = pass_math_s["school_name"].value_counts()
#print(pms_s)

#Get the count of passing reading scores grouped by school
pass_reading_s = school_data_complete[school_data_complete["reading_score"] >= 70]
prs_s = pass_reading_s["school_name"].value_counts()
#print(prs_s)

#Calculate the % passing math and save it to a variable
pmpf = pms_s/num_students*100

#Calculate the % passing reading and save it to a variable
prpf = prs_s/num_students*100

sd_df = school_data_complete[["school_name", "type", "size", "budget", "math_score", 
                              "reading_score"]].set_index("school_name")

#calculate the per student budget
psb = round(sd_df["budget"]/sd_df["size"], 2)
sd_df["Per Student Budget"] = psb
#sd_df

#Add % passing math to a column in the dataframe
sd_df["% Passing Math"] = pd.Series(pmpf)

#Add % passing reading to a column in the dataframe
sd_df["% Passing Reading"] = pd.Series(prpf)

#Populate the overall passing column
sd_df["% Overall Passing Rate"] = (sd_df["% Passing Reading"]+sd_df["% Passing Math"])/2

#Rename fields, sort, and print the final dataframe
sd_df=sd_df.groupby(["school_name", "type"]).mean()
sd_df = sd_df.sort_values(["% Overall Passing Rate"], ascending=False)
sd_df = sd_df.rename(columns={"size":"Total Students", "budget":"Total School Budget", "math_score":"Average Math Score",
                             "reading_score":"Average Reading Score"}, 
                     index={"school_name":"School", "type":"School Type"})

sd_df["Total Students"] = sd_df["Total Students"].astype(np.int64)
sd_df["Total School Budget"] = sd_df["Total School Budget"].astype(np.float)

sd_df_final_numeric = sd_df.reset_index()
#print(sd_df_final_numeric.info())

#Create a copy of the dataframe and convert some of the data into strings
sd_df_string = sd_df.copy()
sd_df_string ["Total School Budget"] = sd_df_string ["Total School Budget"].map("${:,}".format)
sd_df_string ["Per Student Budget"] = sd_df_string ["Per Student Budget"].map("${:,}".format)
#sd_df_string = sd_df_string.rename(columns={"school_name":"School"})
#sd_df_final = sd_df_string .reset_index().set_index("School")
sd_df_final = sd_df_final.rename(columns={"type":"Type"})
sd_df_final
sd_df_final.head()

,Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.0",83.061895,83.975780,$582.0,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.0",83.418349,83.848930,$638.0,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.0",83.839917,84.044699,$609.0,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.0",83.351499,83.816757,$625.0,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.0",83.274201,83.989488,$578.0,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [258]:
sd_df_final.sort_values(by="% Overall Passing Rate", ascending=True).head()

,Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.0",76.842711,80.744686,$637.0,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.0",76.711767,81.158020,$639.0,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.0",76.629414,81.182722,$655.0,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.0",77.072464,80.966394,$650.0,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.0",77.102592,80.746258,$644.0,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [259]:
#Grab the information for calculating the grades
msbg = school_data_complete[["school_name", "grade", "math_score"]]
msbg = msbg.rename(columns={"school_name":"School"})
msbg_grade_9 = msbg[msbg["grade"] == "9th"].groupby(["School"]).mean()["math_score"]
msbg_grade_10 = msbg[msbg["grade"] == "10th"].groupby(["School"]).mean()["math_score"]
msbg_grade_11 = msbg[msbg["grade"] == "11th"].groupby(["School"]).mean()["math_score"]
msbg_grade_12 = msbg[msbg["grade"] == "12th"].groupby(["School"]).mean()["math_score"]

#Add calculations to a new dataframe
msbg_final = pd.DataFrame({"9th Grade":msbg_grade_9, "10th Grade":msbg_grade_10,
                           "11th Grade":msbg_grade_11, "12th Grade":msbg_grade_12})
msbg_final

,9th Grade,10th Grade,11th Grade,12th Grade
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [260]:
#Grab the information for calculating the grades
msbg_reading_score = school_data_complete[["school_name", "grade", "reading_score"]]
msbg_reading_score = msbg_reading_score.rename(columns={"school_name":"School"})
msbg_reading_grade_9 = msbg_reading_score[msbg_reading_score["grade"] == "9th"].groupby(["School"]).mean()["reading_score"]
msbg_reading_grade_10 = msbg_reading_score[msbg_reading_score["grade"] == "10th"].groupby(["School"]).mean()["reading_score"]
msbg_reading_grade_11 = msbg_reading_score[msbg_reading_score["grade"] == "11th"].groupby(["School"]).mean()["reading_score"]
msbg_reading_grade_12 = msbg_reading_score[msbg_reading_score["grade"] == "12th"].groupby(["School"]).mean()["reading_score"]

#Add calculations to a new dataframe
msbg_reading_final = pd.DataFrame({"9th Grade":msbg_reading_grade_9, "10th Grade":msbg_reading_grade_10,
                           "11th Grade":msbg_reading_grade_11, "12th Grade":msbg_reading_grade_12})

msbg_reading_final

,9th Grade,10th Grade,11th Grade,12th Grade
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [261]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

spending_ranges = sd_df_final_numeric[["Per Student Budget", "Average Math Score", "Average Reading Score", 
                               "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].copy()

spending_ranges['Spending Ranges (Per Student)'] = pd.cut(spending_ranges["Per Student Budget"], 
                                                          bins=spending_bins, labels=group_names, include_lowest=True)
spending_ranges.head()

spending_ranges_groups = spending_ranges.sort_values(["% Overall Passing Rate"], ascending=False).groupby(["Spending Ranges (Per Student)"])
spending_ranges_groups.mean()

,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,,
<$585,581.000000,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,604.500000,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,635.166667,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,652.333333,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [262]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
s_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [282]:
sbss = sd_df_final_numeric[["Total Students", "Average Math Score", "Average Reading Score", 
                               "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].copy()

sbss["School Size"] = pd.cut(sbss["Total Students"], bins=size_bins, labels=s_group_names, include_lowest=True)

sbss.head()
sbss_groups = sbss.sort_values(["% Overall Passing Rate"], ascending=False).groupby(["School Size"])
sbss_groups.mean()

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,,
Small (<1000),694.500,83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),1704.400,83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),3657.375,77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [281]:
sbst = sd_df_final_numeric[["type", "Average Math Score", "Average Reading Score", 
                               "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].copy()

#sbst = sbst.rename(index="Type")

sbst_groups = sbst.sort_values(["% Overall Passing Rate"], ascending=False).groupby(["type"]).mean()

sbst_groups

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
